In [2]:
import pandas as pd

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from sqlalchemy import func, inspect
from sqlalchemy import Table, Column, Integer, String, Float, DateTime, MetaData


In [3]:
# Create an engine for the  FemaData.db database

engine = create_engine("sqlite:///../data/data.sqlite", echo=False)
conn = engine.connect()

In [4]:
# Reflect Database into ORM classes
Base = automap_base()
Base.prepare(engine, reflect=True)
meta = MetaData()

In [11]:
filename = "../Trump_Tweets_01012015_12182019.json"
tweets_df = pd.read_json(filename, orient='columns', encoding='utf-8')
tweets_df.head()

,source,text,created_at,retweet_count,favorite_count,is_retweet,id_str
0,Twitter for iPhone,RT @realDonaldTrump: https://t.co/WzLB5s41m3,2019-12-18 05:02:13+00:00,20567,0,1.0,1207164139828854784
1,Twitter for iPhone,.@marcthiessen “Voters say the Democrats are ...,2019-12-18 04:17:19+00:00,13001,52715,0.0,1207152841854509056
2,Twitter for iPhone,"So, if Comey &amp; the top people in the FBI w...",2019-12-18 04:10:40+00:00,24326,86955,0.0,1207151165894471680
3,Twitter for iPhone,....They want to Impeach me (I’m not worried!)...,2019-12-18 04:10:40+00:00,17599,67859,0.0,1207151167173775360
4,Twitter for iPhone,Good marks and reviews on the letter I sent to...,2019-12-18 03:54:22+00:00,28355,115543,0.0,1207147066075549696


In [23]:
retweets = tweets_df[tweets_df["is_retweet"] == 1.0]
own_tweets = tweets_df[tweets_df["is_retweet"] == 0.0]
retweets.head()

,source,text,created_at,retweet_count,favorite_count,is_retweet,id_str
0,Twitter for iPhone,RT @realDonaldTrump: https://t.co/WzLB5s41m3,2019-12-18 05:02:13+00:00,20567,0,1.0,1207164139828854784
25,Twitter for iPhone,RT @T_S_P_O_O_K_Y: It's either incompetence or...,2019-12-16 15:11:04+00:00,12709,0,1.0,1206592587278299136
26,Twitter for iPhone,RT @steph93065: You know why President Trump a...,2019-12-16 15:10:07+00:00,21466,0,1.0,1206592346445492224
27,Twitter for iPhone,RT @RudyGiuliani: Dem’s impeachment for innoce...,2019-12-16 15:09:35+00:00,26859,0,1.0,1206592211812532224
28,Twitter for iPhone,"RT @RudyGiuliani: Witness Yuri Lutsenko, inher...",2019-12-16 15:09:28+00:00,24463,0,1.0,1206592182993506304


In [24]:
own_tweets.head()

,source,text,created_at,retweet_count,favorite_count,is_retweet,id_str
1,Twitter for iPhone,.@marcthiessen “Voters say the Democrats are ...,2019-12-18 04:17:19+00:00,13001,52715,0.0,1207152841854509056
2,Twitter for iPhone,"So, if Comey &amp; the top people in the FBI w...",2019-12-18 04:10:40+00:00,24326,86955,0.0,1207151165894471680
3,Twitter for iPhone,....They want to Impeach me (I’m not worried!)...,2019-12-18 04:10:40+00:00,17599,67859,0.0,1207151167173775360
4,Twitter for iPhone,Good marks and reviews on the letter I sent to...,2019-12-18 03:54:22+00:00,28355,115543,0.0,1207147066075549696
5,Twitter for iPhone,"Democrat “leadership,” despite their denials, ...",2019-12-17 23:48:29+00:00,21740,88682,0.0,1207085185579261952


In [25]:
own_tweets = own_tweets.drop(columns=['is_retweet'])
own_tweets.head()

,source,text,created_at,retweet_count,favorite_count,id_str
1,Twitter for iPhone,.@marcthiessen “Voters say the Democrats are ...,2019-12-18 04:17:19+00:00,13001,52715,1207152841854509056
2,Twitter for iPhone,"So, if Comey &amp; the top people in the FBI w...",2019-12-18 04:10:40+00:00,24326,86955,1207151165894471680
3,Twitter for iPhone,....They want to Impeach me (I’m not worried!)...,2019-12-18 04:10:40+00:00,17599,67859,1207151167173775360
4,Twitter for iPhone,Good marks and reviews on the letter I sent to...,2019-12-18 03:54:22+00:00,28355,115543,1207147066075549696
5,Twitter for iPhone,"Democrat “leadership,” despite their denials, ...",2019-12-17 23:48:29+00:00,21740,88682,1207085185579261952


In [16]:
tweets = Table(
   'tweets', meta, 
    Column('source',String), 
    Column('text', String), 
    Column('created_at', DateTime),
    Column('retweet_count', Integer),
    Column('favorite_count', Integer),
    Column('is_retweet', Float),
    Column('id_str', Integer, primary_key = True)
)


In [17]:
# # - Don't really need this data duplication.  
# # Column is_retweet: 0 (zero) == No, 1 == yes

# no_retweets = Table(
#    'no_retweets', meta, 
#     Column('source',String), 
#     Column('text', String), 
#     Column('created_at', DateTime),
#     Column('retweet_count', Integer),
#     Column('favorite_count', Integer),
#     Column('is_retweet', Float),
#     Column('id_str', Integer, primary_key = True)
# )


In [30]:
#- bind engine, then drop all tables
meta.bind = engine
meta.drop_all()

In [31]:
meta.create_all()

In [32]:
own_tweets.dtypes

source                         object
text                           object
created_at        datetime64[ns, UTC]
retweet_count                   int64
favorite_count                  int64
id_str                          int64
dtype: object

In [33]:
own_tweets.to_sql('tweets', conn, if_exists='append', index=False, index_label="id_str")

IntegrityError: (sqlite3.IntegrityError) UNIQUE constraint failed: tweets.id_str
[SQL: INSERT INTO tweets (source, text, created_at, retweet_count, favorite_count, id_str) VALUES (?, ?, ?, ?, ?, ?)]
[parameters: (('Twitter for iPhone', '.@marcthiessen  “Voters say the Democrats are trying to take away their votes. Pelosi was right the first time. Impeachment has backfired thoroughly on the Democrats. How can you Impeach the President for going to the courts?”', '2019-12-18 04:17:19.000000', 13001, 52715, 1207152841854509056), ('Twitter for iPhone', 'So, if Comey &amp; the top people in the FBI were dirty cops and cheated on the FISA Court, wouldn’t all of these phony cases have to be overturned or dismissed? They went after me with the Fake Dossier, paid for by Crooked Hillary &amp; the DNC, which they illegally presented to FISA...', '2019-12-18 04:10:40.000000', 24326, 86955, 1207151165894471680), ('Twitter for iPhone', '....They want to Impeach me (I’m not worried!), and yet they we

# Load no retweets data

No need really.  Retweets appear to be marked as 1 in is_retweet column.

In [ ]:
# filename = "../no_retweets_20150101_20191218.json"
# retweets_df = pd.read_json(filename, orient='columns', encoding='utf-8')


In [ ]:
# retweets_df.shape

In [ ]:
# # Make sure the table is clean...
# engine.execute("delete from no_retweets")

In [ ]:
# retweets_df.to_sql('no_retweets', conn, if_exists='append', index=False, index_label="id_str")

In [ ]:
# Compact/compress db after working on it.
engine.execute("VACUUM")

In [22]:
conn.close

<bound method Connection.close of <sqlalchemy.engine.base.Connection object at 0x000002AD817DF550>>